In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import xgboost as xgb
import lightgbm as lgbm
import catboost
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score
from IPython.display import display
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import optuna
from optuna.samplers import TPESampler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

# Loading Data

In [2]:
BASE_DIR = Path("/kaggle/input/playground-series-s3e2/")

train = pd.read_csv(BASE_DIR / "train.csv").drop(columns="id")
test = pd.read_csv(BASE_DIR / "test.csv").drop(columns="id")

# Preprocessing

In [3]:
df = pd.concat([train.drop(columns=["stroke"]), test], axis=0).reset_index(drop=True)
df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,Male,28.0,0,0,Yes,Private,Urban,79.53,31.1,never smoked
1,Male,33.0,0,0,Yes,Private,Rural,78.44,23.9,formerly smoked
2,Female,42.0,0,0,Yes,Private,Rural,103.00,40.3,Unknown
3,Male,56.0,0,0,Yes,Private,Urban,64.87,28.8,never smoked
4,Female,24.0,0,0,No,Private,Rural,73.36,28.8,never smoked


# Feature Engineering

In [4]:
df['morbid'] = np.where(df.bmi>40,1,0)
df['obese'] = np.where(df.bmi>30,1,0)
# df.head()

def feature_risk_factors(df):
    df["risk_factors"] = df[[
        "avg_glucose_level", "age", "bmi", 
        "hypertension", "heart_disease", 
        "smoking_status"
    ]].apply(
        lambda x: \
        0 + (1 if x.avg_glucose_level > 99 else 0) + \
        (1 if x.age > 45 else 0) + (1 if x.bmi > 24.99 else 0) + \
        (1 if x.hypertension == 1 else 0) + \
        (1 if x.heart_disease == 1 else 0) + \
        (1 if x.smoking_status in ["formerly smoked", "smokes"] else 0),
        axis=1
    )
    return df
feature_risk_factors(df)

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,morbid,obese,risk_factors
0,Male,28.0,0,0,Yes,Private,Urban,79.53,31.1,never smoked,0,1,1
1,Male,33.0,0,0,Yes,Private,Rural,78.44,23.9,formerly smoked,0,0,1
2,Female,42.0,0,0,Yes,Private,Rural,103.00,40.3,Unknown,1,1,2
3,Male,56.0,0,0,Yes,Private,Urban,64.87,28.8,never smoked,0,0,2
4,Female,24.0,0,0,No,Private,Rural,73.36,28.8,never smoked,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25503,Female,27.0,0,0,No,Private,Urban,75.77,17.6,never smoked,0,0,0
25504,Male,49.0,0,0,Yes,Private,Urban,102.91,26.7,Unknown,0,0,3
25505,Female,3.0,0,0,No,children,Rural,104.04,18.3,Unknown,0,0,1
25506,Male,31.0,0,0,Yes,Private,Urban,82.41,28.7,never smoked,0,0,1


In [ ]:
# df["diabetes"] = df.avg_glucose_level.map(diabetes_indicator)
# df.head(3)

# One hot encoding

In [5]:
df = pd.get_dummies(df)
df.head(3)

,age,hypertension,heart_disease,avg_glucose_level,bmi,morbid,obese,risk_factors,gender_Female,gender_Male,...,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,Residence_type_Urban,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,28.0,0,0,79.53,31.1,0,1,1,0,1,...,0,1,0,0,0,1,0,0,1,0
1,33.0,0,0,78.44,23.9,0,0,1,0,1,...,0,1,0,0,1,0,0,1,0,0
2,42.0,0,0,103.00,40.3,1,1,2,1,0,...,0,1,0,0,1,0,1,0,0,0


# Preprocessing v2

In [6]:
X = df.iloc[:-len(test), :]
test_new = df.iloc[-len(test):, :]
y = train.stroke

In [11]:
inputs = layers.Input(shape=(24,))
x = layers.Dense(1024, activation="relu")(inputs)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.3)(x)

x = layers.Dense(512, activation="relu")(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.2)(x)

x = layers.Dense(160, activation="relu")(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.7)(x)

x = layers.Dense(32, activation="relu")(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.1)(x)

outputs = layers.Dense(1, activation="sigmoid")(x)

keras_model = keras.Model(inputs=inputs, outputs=outputs)

optim = keras.optimizers.Adam(learning_rate= 0.00513)

keras_model.compile(optimizer=optim,
                   loss=keras.losses.binary_crossentropy,
                   metrics=[keras.metrics.AUC()])

In [12]:
early_stopping = keras.callbacks.EarlyStopping(
                patience=20,
                min_delta=0.001,
                monitor="val_auc",
                restore_best_weights=True,
                )

reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=5, min_lr=0.001)

In [13]:
history = keras_model.fit(X, y, validation_split=0.1, epochs=44, callbacks=[reduce_lr])

Epoch 1/44
431/431 [==============================] - 7s 13ms/step - loss: 0.1950 - auc_1: 0.7432 - val_loss: 0.1290 - val_auc_1: 0.8948
Epoch 2/44
431/431 [==============================] - 4s 10ms/step - loss: 0.1406 - auc_1: 0.8413 - val_loss: 0.1181 - val_auc_1: 0.9036
Epoch 3/44
431/431 [==============================] - 4s 10ms/step - loss: 0.1351 - auc_1: 0.8620 - val_loss: 0.1227 - val_auc_1: 0.9000
Epoch 4/44
431/431 [==============================] - 4s 10ms/step - loss: 0.1364 - auc_1: 0.8573 - val_loss: 0.1189 - val_auc_1: 0.8956
Epoch 5/44
431/431 [==============================] - 4s 10ms/step - loss: 0.1337 - auc_1: 0.8652 - val_loss: 0.1176 - val_auc_1: 0.8936
Epoch 6/44
431/431 [==============================] - 4s 10ms/step - loss: 0.1350 - auc_1: 0.8606 - val_loss: 0.1194 - val_auc_1: 0.9001
Epoch 7/44
431/431 [==============================] - 4s 10ms/step - loss: 0.1350 - auc_1: 0.8587 - val_loss: 0.1219 - val_auc_1: 0.8904
Epoch 8/44
431/431 [=====================

In [ ]:
# keras_model.fit(X, y, epochs=44)

In [14]:
y_pred = keras_model.predict(test_new)[:, 0]
y_pred

array([0.02591196, 0.1479151 , 0.00030077, ..., 0.00025523, 0.00150332,
       0.00015175], dtype=float32)

In [15]:
# since we dropped id column from our test dataframe earlier but we need it for submission file so we're reloading it
test_idx = pd.read_csv(BASE_DIR / "test.csv").id
test_idx

0        15304
1        15305
2        15306
3        15307
4        15308
         ...  
10199    25503
10200    25504
10201    25505
10202    25506
10203    25507
Name: id, Length: 10204, dtype: int64

In [16]:
submission = pd.DataFrame(data={"id": test_idx, "stroke": y_pred})
submission.head()

,id,stroke
0,15304,0.025912
1,15305,0.147915
2,15306,0.000301
3,15307,0.057842
4,15308,0.007878


In [17]:
submission.to_csv("submission.csv", index=False)